[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/01_Presets_and_Scenarios.ipynb)


In [ ]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


# Presets and Scenario Validation

Learn how to inspect clinic-safe presets and validate custom scenarios.


In [ ]:
import json
from iints.presets import load_presets

from iints.validation import validate_scenario_dict, scenario_warnings

presets = load_presets()
[(p["name"], p["description"]) for p in presets]


In [ ]:
preset = next(p for p in presets if p["name"] == "midnight_crash")
scenario = preset["scenario"]

model = validate_scenario_dict(scenario)
warnings = scenario_warnings(model)
model.model_dump(), warnings


## Create a custom scenario


In [ ]:
custom = {
    "scenario_name": "Custom Breakfast",
    "scenario_version": "1.0",
    "description": "Small breakfast with delayed absorption",
    "stress_events": [
        {
            "start_time": 30,
            "event_type": "meal",
            "value": 35,
            "absorption_delay_minutes": 10,
            "duration": 60,
        }
    ],
}

validate_scenario_dict(custom).model_dump()


In [ ]:
from pathlib import Path
output_path = Path("results/custom_breakfast.json")
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(custom, indent=2))
